# The Pst class

The `pst_handler` module contains the `Pst` class for dealing with pest control files.  It relies heavily on `pandas` to deal with tabular sections, such as parameters, observations, and prior information.  

In [1]:
from __future__ import print_function
import os
import numpy as np
from pyemu import Pst

We need to pass the name of a pest control file to instantiate:

In [2]:
pst_name = os.path.join("henry","pest.pst")
p = Pst(pst_name)

All of the relevant parts of the pest control file are attributes of the `pst` class with the same name:



In [3]:
p.parameter_data

,parnme,partrans,parchglim,parval1,parlbnd,parubnd,pargp,scale,offset,dercom
0,global_k,log,factor,200,150.00,250.00,m,1,0,1
1,mult1,log,factor,1,0.75,1.25,m,1,0,1
2,mult2,log,factor,1,0.50,2.00,m,1,0,1
3,kr01c01,log,factor,1,0.10,10.00,p,1,0,1
4,kr01c02,log,factor,1,0.10,10.00,p,1,0,1
5,kr01c03,log,factor,1,0.10,10.00,p,1,0,1
6,kr01c04,log,factor,1,0.10,10.00,p,1,0,1
7,kr01c05,log,factor,1,0.10,10.00,p,1,0,1
8,kr01c06,log,factor,1,0.10,10.00,p,1,0,1
9,kr01c07,log,factor,1,0.10,10.00,p,1,0,1


In [4]:
p.observation_data

,obsnme,obsval,weight,obgnme
0,h_obs01_1,5.139620e-02,152.14580,head
1,h_obs01_2,2.215620e-02,0.00000,head
2,h_obs02_1,4.687940e-02,152.14580,head
3,h_obs02_2,2.085280e-02,0.00000,head
4,h_obs03_1,3.658410e-02,152.14580,head
5,h_obs03_2,1.950220e-02,0.00000,head
6,h_obs04_1,2.754160e-02,152.14580,head
7,h_obs04_2,1.694640e-02,0.00000,head
8,h_obs05_1,2.638160e-02,152.14580,head
9,h_obs05_2,7.545570e-03,0.00000,head


In [5]:
p.prior_information

,equation,obgnme,pilbl,weight
0,1.0 * log(mult1) = 0.000000,regul_m,mult1,1
1,1.0 * log(kr01c01) = 0.0,regul_p,kr01c01,1
2,1.0 * log(kr01c02) = 0.0,regul_p,kr01c02,1
3,1.0 * log(kr01c03) = 0.0,regul_p,kr01c03,1
4,1.0 * log(kr01c04) = 0.0,regul_p,kr01c04,1
5,1.0 * log(kr01c05) = 0.0,regul_p,kr01c05,1
6,1.0 * log(kr01c06) = 0.0,regul_p,kr01c06,1
7,1.0 * log(kr01c07) = 0.0,regul_p,kr01c07,1
8,1.0 * log(kr01c08) = 0.0,regul_p,kr01c08,1
9,1.0 * log(kr01c09) = 0.0,regul_p,kr01c09,1


A residual file (`.rei` or `res`) can also be passed to the `resfile` argument at instantiation to enable some simple residual analysis and weight adjustments.  If the residual file is in the same directory as the pest control file and has the same base name, it will be accessed automatically:


In [6]:
p.res

,name,group,measured,modelled,residual,weight
name,,,,,,
h_obs01_1,h_obs01_1,head,0.051396,0.080402,-0.029006,152.1458
h_obs01_2,h_obs01_2,head,0.022156,0.036898,-0.014742,0.0000
h_obs02_1,h_obs02_1,head,0.046879,0.069121,-0.022241,152.1458
h_obs02_2,h_obs02_2,head,0.020853,0.034311,-0.013458,0.0000
h_obs03_1,h_obs03_1,head,0.036584,0.057722,-0.021138,152.1458
h_obs03_2,h_obs03_2,head,0.019502,0.031408,-0.011905,0.0000
h_obs04_1,h_obs04_1,head,0.027542,0.045770,-0.018229,152.1458
h_obs04_2,h_obs04_2,head,0.016946,0.027337,-0.010391,0.0000
h_obs05_1,h_obs05_1,head,0.026382,0.031442,-0.005060,152.1458


The `pst` class has some `@decorated` convience methods related to the residuals:

In [7]:
print(p.phi,p.phi_components)

1855.68743783 {'head': 1658.6292168686423, 'conc': 197.05822096106502}


Some additional `@decorated` convience methods:

In [8]:
print(p.npar,p.nobs,p.nprior)

603 75 601


In [9]:
print(p.par_groups,p.obs_groups)

['m', 'p'] <map object at 0x107a2ee10>


In [10]:
print(type(p.par_names)) # all parameter names
print(type(p.adj_par_names)) # adjustable parameter names
print(type(p.obs_names)) # all observation names
print(type(p.nnz_obs_names)) # non-zero weight observations

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>


The "control_data" section of the pest control file is accessible in the `Pst.control_data` attribute:

In [11]:
print('jacupdate = {0}'.format(p.control_data.jacupdate))
print('numlam = {0}'.format(p.control_data.numlam))
p.control_data.numlam = 100
print('numlam has been changed to --> {0}'.format(p.control_data.numlam))

jacupdate = 999
numlam = 10
numlam has been changed to --> 100


The `Pst` class also exposes a method to get a new `Pst` instance with a subset of parameters and or obseravtions.  Note this method does not propogate prior information to the new instance:

In [12]:
pnew = p.get(p.par_names[:10],p.obs_names[-10:])
print(pnew.prior_information)

Empty DataFrame
Columns: [obgnme, pilbl]
Index: []


You can also write a pest control file with altered parameters, observations, and/or prior information:

In [13]:
pnew.write("test.pst")

Some other methods in `Pst` include:

In [14]:
# add preferred value regularization with weights proportional to parameter bounds
pnew.zero_order_tikhonov()
pnew.prior_information

,equation,obgnme,pilbl,weight
pilbl,,,,
global_k,1.0 * log(global_k) = 2.301030E+00,regul,global_k,4.507576
mult1,1.0 * log(mult1) = 0.000000E+00,regul,mult1,4.507576
mult2,1.0 * log(mult2) = 0.000000E+00,regul,mult2,1.660964
kr01c01,1.0 * log(kr01c01) = 0.000000E+00,regul,kr01c01,0.500000
kr01c02,1.0 * log(kr01c02) = 0.000000E+00,regul,kr01c02,0.500000
kr01c03,1.0 * log(kr01c03) = 0.000000E+00,regul,kr01c03,0.500000
kr01c04,1.0 * log(kr01c04) = 0.000000E+00,regul,kr01c04,0.500000
kr01c05,1.0 * log(kr01c05) = 0.000000E+00,regul,kr01c05,0.500000
kr01c06,1.0 * log(kr01c06) = 0.000000E+00,regul,kr01c06,0.500000


In [18]:
# add preferred value regularization with unity weights
pnew.zero_order_tikhonov(parbounds=False)
pnew.prior_information

,equation,obgnme,pilbl,weight
0,1.0 * log(global_k) = 2.301030E+00,regul,global_k,1
1,1.0 * log(mult1) = 0.000000E+00,regul,mult1,1
2,1.0 * log(mult2) = 0.000000E+00,regul,mult2,1
3,1.0 * log(kr01c01) = 0.000000E+00,regul,kr01c01,1
4,1.0 * log(kr01c02) = 0.000000E+00,regul,kr01c02,1
5,1.0 * log(kr01c03) = 0.000000E+00,regul,kr01c03,1
6,1.0 * log(kr01c04) = 0.000000E+00,regul,kr01c04,1
7,1.0 * log(kr01c05) = 0.000000E+00,regul,kr01c05,1
8,1.0 * log(kr01c06) = 0.000000E+00,regul,kr01c06,1
9,1.0 * log(kr01c07) = 0.000000E+00,regul,kr01c07,1


Some more `res` functionality

In [19]:
# adjust observation weights to account for residual phi components
#pnew = p.get()
print(p.phi, p.nnz_obs, p.phi_components)
p.adjust_weights_resfile()
print(p.phi, p.nnz_obs, p.phi_components)

1855.68743783 36 {'head': 1658.6292168686423, 'conc': 197.05822096106502}
36.0 36 {'head': 21.0, 'conc': 15.000000000000004}


adjust observation weights by an arbitrary amount by groups:

In [20]:
print(p.phi, p.nnz_obs, p.phi_components)
grp_dict = {"head":100}
p.adjust_weights(obsgrp_dict=grp_dict)
print(p.phi, p.nnz_obs, p.phi_components)

36.0 36 {'head': 21.0, 'conc': 15.000000000000004}
115.0 36 {'head': 99.999999999999972, 'conc': 15.000000000000004}


adjust observation weights by an arbitrary amount by individual observations:

In [21]:
print(p.phi, p.nnz_obs, p.phi_components)
obs_dict = {"h_obs01_1":25}
p.adjust_weights(obs_dict=obs_dict)
print(p.phi, p.nnz_obs, p.phi_components)

115.0 36 {'head': 99.999999999999972, 'conc': 15.000000000000004}
138.825807011 36 {'head': 123.82580701146588, 'conc': 15.000000000000004}


setup weights inversely proportional to the observation values

In [22]:
p.adjust_weights_resfile()
print(p.phi, p.nnz_obs, p.phi_components)
p.proportional_weights(fraction_stdev=0.1,wmax=20.0)
print(p.phi, p.nnz_obs, p.phi_components)

36.0 36 {'head': 21.0, 'conc': 15.0}
222.969965622 36 {'head': 28.660869806976095, 'conc': 194.30909581453392}
